In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/My Drive/methylation')

Mounted at /content/drive/


In [ ]:
!pip install h5py==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.9 MB/s eta 0:00:00
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# %tensorflow_version 1
#Start tensorboard
%tensorboard — logdir logs
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter('ignore')

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] {serve,dev} ...
tensorboard: error: invalid choice: '—' (choose from 'serve', 'dev')

In [ ]:
seed = 0
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)

In [ ]:
%matplotlib inline  

import imp
import numpy as np
import os
import keras
import keras.backend
import keras.models
import sklearn
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras import regularizers
from keras.models import Model,Sequential
from keras.layers import Input,Dense,Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.utils import plot_model
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import keras.models as mod
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import TensorBoard
from keras.utils.vis_utils import plot_model
from sklearn.externals import joblib


In [ ]:
%tensorflow_version 1
seed=0

In [ ]:
seed=0      #Initially 0
np.random.seed(seed)
df = pd.read_csv('genelevelMethyl.csv', index_col=0)
df = df.reindex(sorted(df.columns), axis=1)
df=df.T
print("CNV Matrix Before:", df.shape)

df2=pd.read_csv('BRCA_clinicalMatrix.gz',compression='gzip',sep='\t',index_col=0)
df2.index = pd.Series({x: x.replace('-', '.') for x in df2.index})
print("Clinical Data Before:", df2.shape)
k=df2.columns.get_loc('PAM50Call_RNAseq')
df2=df2[df2.iloc[:,k].isna()==False]

commonIndexes=np.sort(list(set(df2.index).intersection(set(df.index)))) 
df=df.loc[commonIndexes]
df2=df2.loc[commonIndexes] 

print("CNV After:", df.shape)
print("Clinical Data After:", df2.shape)
print("CNV and Clinical Data corresponds to same Patients: ", False if False in (df.index==df2.index) else True)
unique_elements, counts_elements = np.unique(df2.iloc[:,k], return_counts=True)
print("Labels", unique_elements, counts_elements)

CNV Matrix Before: (888, 22382)
Clinical Data Before: (1247, 202)
CNV After: (620, 22382)
Clinical Data After: (620, 202)
CNV and Clinical Data corresponds to same Patients:  True
Labels ['Basal' 'Her2' 'LumA' 'LumB' 'Normal'] [ 87  31 288 127  87]


In [ ]:
genes_52 = ['CFH',
 'RHBDL1',
 'OR51B2',
 'OR10G9',
 'MIR592',
 'OR5B17',
 'DEF6',
 'ZNF671',
 'FLACC1',
 'GBP4',
 'FBXO47',
 'SERPINA3',
 'OR8D4',
 'DGKB',
 'SYNGR2',
 'CHAD',
 'VTRNA1-2',
 'OR8J3',
 'AGR3',
 'RP11-12M5.1',
 'RPL13AP',
 'ARHGAP40',
 'RP4-761J14.9',
 'AC073508.1',
 'AOX1',
 'PTPRQ',
 'CYP2F1',
 'PPP2R3A',
 'CARD6',
 'CTD-2370N5.3',
 'EMBP1',
 'SNORD32B',
 'RP11-344P13.6',
 'FJX1',
 'OR2T4',
 'FMOD',
 'MT1DP',
 'ARHGDIB',
 'RP11-159D12.2',
 'MESTIT1_1',
 'PRELP',
 'OR51B6',
 'SLC2A5',
 'OR10S1',
 'CTD-2298J14.2',
 'SYNPR-AS1',
 'TUSC7',
 'KLK3',
 'TXLNB',
 'IFI27',
 'MNDA',
 'KRTAP19-1']

In [ ]:
X=np.array(df[list(set(genes_52))])
X.shape

(620, 52)

In [ ]:
Y_temp=np.array(df2.values[:,k])
print("TARGET LABEL SIZE BEFORE: ", Y_temp.shape)
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(Y_temp)
print("TARGET LABEL SIZE BEFORE: ", Y.shape)
print("Labels before and after:", Y_temp[0:9], Y[0:9])
le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(le_name_mapping)

TARGET LABEL SIZE BEFORE:  (620,)
TARGET LABEL SIZE BEFORE:  (620,)
Labels before and after: ['Normal' 'LumA' 'LumA' 'LumA' 'LumA' 'LumB' 'LumA' 'Basal' 'LumA'] [4 2 2 2 2 3 2 0 2]
{'Basal': 0, 'Her2': 1, 'LumA': 2, 'LumB': 3, 'Normal': 4}


In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)
Y = Y.reshape(len(Y), 1)
Y_onehot = Y
print(Y_onehot.shape)

(620, 1)


In [ ]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from keras.layers import Softmax

np.random.seed(seed)

input = Input(shape=(X.shape[1],))
hidden1 = Dense(40, activation='relu')(input)
dropout1 = Dropout(0.3, seed=seed)(hidden1)
hidden2 = Dense(10, activation='relu')(dropout1)
dropout2 = Dropout(0.3, seed=seed)(hidden2)
hidden3 = Dense(5, activation=None)(dropout2)
softmax = Softmax()(hidden3)
classifier = Model(inputs=input, outputs=softmax)
print(classifier.summary())

adam=Adam(lr=0.002)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 52)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 40)                2120      
_________________________________________________________________
dropout_3 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                410       
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 55        
_________________________________________________________________
softmax_2 (Softmax)          (None, 5)                 0   

In [ ]:
total = 0
i = 0
for train_idx, test_idx in skf.split(X, Y_onehot):
  X_train = X[train_idx]
  Y_train = Y_onehot[train_idx]
  X_test = X[test_idx]
  Y_test = Y_onehot[test_idx]

  sm=SMOTE(random_state=seed)
  X_train_res,Y_train_res=sm.fit_sample(X_train,Y_train)

  classifier = Model(inputs=input, outputs=softmax)

  classifier.compile(optimizer=adam, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

  classifier.fit(X_train_res, Y_train_res, epochs=100, batch_size=16, verbose=0)
  y_pred = classifier.predict(X_test)
  print("\n Fold " + str(i) + " Accuracy: ", accuracy_score(Y_test, y_pred.argmax(axis=-1)))
  print("\n Confusion Matrix: ")
  print(confusion_matrix(Y_test, y_pred.argmax(axis=-1)))
  total += accuracy_score(Y_test, y_pred.argmax(axis=-1))
  i += 1

print('Average Accuracy: ', total/5)


 Fold 0 Accuracy:  0.7903225806451613

 Confusion Matrix: 
[[17  0  0  0  0]
 [ 1  3  0  2  0]
 [ 0  0 41 12  4]
 [ 0  0  6 20  0]
 [ 0  0  1  0 17]]

 Fold 1 Accuracy:  0.75

 Confusion Matrix: 
[[16  1  0  0  0]
 [ 1  3  0  2  0]
 [ 0  0 45 10  2]
 [ 0  1 14 11  0]
 [ 0  0  0  0 18]]

 Fold 2 Accuracy:  0.8306451612903226

 Confusion Matrix: 
[[18  0  0  0  0]
 [ 1  4  0  1  0]
 [ 0  0 42 15  1]
 [ 0  0  2 23  0]
 [ 0  0  1  0 16]]

 Fold 3 Accuracy:  0.8548387096774194

 Confusion Matrix: 
[[17  0  0  1  0]
 [ 0  5  0  0  1]
 [ 0  0 48  6  4]
 [ 0  0  5 20  0]
 [ 1  0  0  0 16]]

 Fold 4 Accuracy:  0.8467741935483871

 Confusion Matrix: 
[[16  0  1  0  0]
 [ 0  7  0  0  0]
 [ 0  0 44 10  4]
 [ 0  0  3 22  0]
 [ 1  0  0  0 16]]
Average Accuracy:  0.8145161290322582
